**Задание 1**  
Напишите функцию, которая возвращает название валюты (поле 'Name') с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

In [5]:
from libs.exchange import Rate

def get_most_expensive_valute():
    try:
        rates = Rate().exchange_rates()
    except:
        return
    
    # ищем максимум
    maximum = None
    for val in rates.values():
        if maximum is None:
            maximum = val['Value']
        else:
            if maximum < val['Value']:
                maximum = val['Value']

    if maximum is None:
        return
    
    # а вдруг, валют с максимальной стоимостью несколько?
    # составляем список валют с макс. значением
    lst = []
    for val in rates.values():
        if val['Value'] == maximum:
            lst.append(val['Name'])
    return { "max": maximum, "valutes": lst }

result = get_most_expensive_valute()
if result is None:
    print("Максимум не найден, видимо, что-то с данными")
else:
    if len(result["valutes"]) == 1:
        print(f'Самая дорогая валюта \'{result["valutes"][0]}\' имеет стоимость {result["max"]}.')
    else:
        print(f'Самых дорогих валют несколько, стоит каждая из них {result["max"]}.\n' +
              f'Вот их список: {result["valutes"]}.')


Самая дорогая валюта 'СДР (специальные права заимствования)' имеет стоимость 99.4937.


**Задание 2**  
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [9]:
import requests

class Rate:
    """
    
    """
    def __init__(self, format_='value', diff=False):
        self.format = format_
        self.diff = diff
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                if self.diff:
                    return response[currency]['Value'] - response[currency]['Previous']
                else:
                    return response[currency]['Value']
        
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')

r1 = Rate()
r2 = Rate(diff=True)
try:
    print(f"Валюта           : {'Курс':8} {'Дельта':8}")
    print(f"Евро             : {r1.eur():<8.4} {r2.eur():<8.4}")
    print(f"Доллар           : {r1.usd():<8.4} {r2.usd():<8.4}")
    print(f"Бразильский Реал : {r1.brl():<8.4} {r2.brl():<8.4}")
except:
    print("Что-то пошло не так")

Валюта           : Курс     Дельта  
Евро             : 81.68    0.4503  
Доллар           : 71.71    0.4829  
Бразильский Реал : 13.45    0.1868  


**Задание 3**  
Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: "Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла"). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять). Выполните проверку для 20 аккредитаций дизайнера Елены.  
Для проверки используйте код:  
```python
elena = Designer('Елена', seniority=0, awards=2)
for i in range(20):
    elena.check_if_it_is_time_for_upgrade()
```

In [10]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)
    
    def check_if_it_is_time_for_upgrade(self):
        pass

    
class Designer(Employee):
    def __init__(self, name, seniority, awards):
        super().__init__(name, seniority)
        self.seniority += awards * 2          # +2 балла за премию, кол-во не меняется
        
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все дизайнеры проходят аккредитацию
        self.seniority += 1
        
        # условие повышения сотрудника из презентации
        if self.seniority % 7 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()
    
    
elena = Designer('Елена', seniority=0, awards=2)
for i in range(20):
    elena.check_if_it_is_time_for_upgrade()

Елена 1
Елена 1
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 4
Елена 4
Елена 4
Елена 4
